In [1]:
import librosa
import tensorflow as tf
import numpy as np
from ai_edge_litert.interpreter import Interpreter

2025-05-16 01:22:23.493467: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 01:22:23.506376: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747372943.521114   35124 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747372943.524922   35124 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747372943.536625   35124 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
encoder = "/home/dylenthomas/whisperProject/whisper/.model/WhisperEncoder.tflite"
decoder = "/home/dylenthomas/whisperProject/whisper/.model/WhisperDecoder.tflite"
audio = "/home/dylenthomas/whisperProject/whisper/test.wav"

In [3]:
test = Interpreter(model_path="/home/dylenthomas/whisperProject/whisper/.model/whisper_english.tflite")

In [4]:
from utils.WhisperProcessor import offlineWhisperProcessor
import torch

signal, _ = librosa.load(audio, sr=16000, mono=True)

processor = offlineWhisperProcessor("/home/dylenthomas/whisperProject/whisper/utils/preprocessor_config.json",
                                    "/home/dylenthomas/whisperProject/whisper/utils/tokenizer_config.json",
                                    "/home/dylenthomas/whisperProject/whisper/utils/vocab.json")
signal_test = torch.Tensor(signal)

waveform_features = processor.extract_features(signal_test)
print((waveform_features))

tensor([[[-0.0858,  0.2856,  0.3675,  ..., -1.0069, -1.0069, -1.0069],
         [ 0.3361,  0.3777,  0.2334,  ..., -1.0069, -1.0069, -1.0069],
         [ 0.3076,  0.4046,  0.3225,  ..., -1.0069, -1.0069, -1.0069],
         ...,
         [-0.4106, -0.3675, -0.4100,  ..., -1.0069, -1.0069, -1.0069],
         [-0.4572, -0.3279, -0.4158,  ..., -1.0069, -1.0069, -1.0069],
         [-0.3721, -0.3522, -0.4597,  ..., -1.0069, -1.0069, -1.0069]]])


In [5]:
encoder_interpreter = Interpreter(model_path=encoder)
decoder_interpreter = Interpreter(model_path=decoder)

encoder_input_details = encoder_interpreter.get_input_details()
encoder_output_details = encoder_interpreter.get_output_details()
decoder_input_details = decoder_interpreter.get_input_details()
decoder_output_details = decoder_interpreter.get_output_details()

def print_details(details, type):
    print("-"*10 + f" {type} details ".upper() + "-"*10)
    for detail in details:
        print(detail)

print_details(encoder_input_details, "encoder input")
print_details(encoder_output_details, "encoder output")
print_details(decoder_input_details, "decoder input")
print_details(decoder_output_details, "decoder output")

---------- ENCODER INPUT DETAILS ----------
{'name': 'audio', 'index': 0, 'shape': array([   1,   80, 3000], dtype=int32), 'shape_signature': array([   1,   80, 3000], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
---------- ENCODER OUTPUT DETAILS ----------
{'name': 'k_cache', 'index': 387, 'shape': array([   4,    6,   64, 1500], dtype=int32), 'shape_signature': array([   4,    6,   64, 1500], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'v_cache', 'index': 420, 'shape': array([   4,    6, 1500,   64], dtype=int32), 'shape_signature': array([   4,    6, 1500,   64], dtype=int32), 'dtype': <class 'numpy.float32'

In [6]:
encoder_interpreter.allocate_tensors()

#set the audio tensor for inference
encoder_interpreter.set_tensor(encoder_input_details[0]["index"], waveform_features)

with tf.device("/CPU:0"):
    #run inference
    encoder_interpreter.invoke()

k_cache = encoder_interpreter.get_tensor(encoder_output_details[0]["index"])
v_cache = encoder_interpreter.get_tensor(encoder_output_details[1]["index"])

print("k_cache shape: {}".format(k_cache.shape))
print("v_cache shape: {}".format(v_cache.shape))

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1747372949.309422   35124 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5520 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


k_cache shape: (4, 6, 64, 1500)
v_cache shape: (4, 6, 1500, 64)


In [ ]:
decoder_interpreter.allocate_tensors()

#set the start (or last predicted token) token
decoder_interpreter.set_tensor(decoder_input_details[0]["index"], np.array([[50258]], np.int32))
#set the starting index
decoder_interpreter.set_tensor(decoder_input_details[1]["index"], np.array([[1]], np.int32)) # seems to like to start at 1
#set the k and v caches for decoding
decoder_interpreter.set_tensor(decoder_input_details[2]["index"], k_cache)
decoder_interpreter.set_tensor(decoder_input_details[3]["index"], v_cache)

#run inference
decoder_interpreter.invoke()

logits = decoder_interpreter.get_tensor(decoder_output_details[0]["index"])
prev_k_cache = decoder_interpreter.get_tensor(decoder_output_details[1]["index"])
prev_v_cache = decoder_interpreter.get_tensor(decoder_output_details[2]["index"])

probabilities = tf.nn.softmax(logits, axis=-1)
val = tf.argmax(probabilities, axis=-1).numpy()
val = val.item()
print(processor.decode_single(val))

# I think the pipeline for this model is you feed the token that came out of the decoder aswell as the encoded signal back in to the decoder (without encoding agian) with self attention until it hits the end of transcript token

### Find another tflite encoder decoder file

In [36]:
not_end = True
i = 1
start_tok = 50258 #<|startoftranscript|>

decoder_interpreter.allocate_tensors()

while not_end:
    decoder_interpreter.set_tensor(decoder_input_details[0]["index"], np.array([[start_tok]], np.int32))
    decoder_interpreter.set_tensor(decoder_input_details[1]["index"], np.array([[i]], np.int32))
    decoder_interpreter.set_tensor(decoder_input_details[2]["index"], k_cache)
    decoder_interpreter.set_tensor(decoder_input_details[3]["index"], v_cache)

    if i > 1:
        decoder_interpreter.set_tensor(decoder_input_details[4]["index"], prev_k_cache)
        decoder_interpreter.set_tensor(decoder_input_details[5]["index"], prev_v_cache)

    logits = decoder_interpreter.get_tensor(decoder_output_details[0]["index"])
    logits = decoder_interpreter.get_tensor(decoder_output_details[0]["index"])
    prev_k_cache = decoder_interpreter.get_tensor(decoder_output_details[1]["index"])
    prev_v_cache = decoder_interpreter.get_tensor(decoder_output_details[2]["index"])

    probabilities = tf.nn.softmax(logits, axis=-1)
    start_tok = tf.argmax(probabilities, axis=-1).numpy().item()
    print(processor.decode_single(val), val)

    if start_tok == 50257:
        not_end = False

 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256
 50256

KeyboardInterrupt: 